In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [3]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_wa.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,WA,98112.0,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN
1,921945,1101807307,4/28/20,1013,PPP,COMPASS HEALTH,4526 FEDERAL AVE Building 4E,EVERETT,WA,98203.0,...,999 Other,6214 Outpatient Care Centers,6214.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,NaN
2,921946,1351807201,4/15/20,1013,PPP,NORTHWEST RESTAURANTS INC,18815 139th Ave NE Suite C,WOODINVILLE,WA,98072.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,NaN
3,921947,2545378910,4/27/21,1013,PPP,BLOODWORKS,921 Terry Ave,Seattle,WA,98104.0,...,999 Other,6219 Other Ambulatory Health Care Services,6219.0,SEATTLE DISTRICT OFFICE,e $5-10 million,250 to 499,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,NaN
4,921948,2664207106,4/11/20,1013,PPP,"MOD SUPER FAST PIZZA, LLC",2035 158th Court NE 0.0,Bellevue,WA,98008.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23750,945694,9825088608,3/26/21,1013,PPS,SOUND OIL CO,6346 Rainier Ave S,Seattle,WA,98118.0,...,999 Other,4543 Direct Selling Establishments,4543.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"6346 Rainier Ave S, Seattle, WA",47.545071,-122.274353,NaN
23751,945695,9830848507,3/12/21,1013,PPS,AEROTECH COATINGS LLC,6021 199th St NE Ste A,Arlington,WA,98223.0,...,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6021 199th St NE Ste A, Arlington, WA",48.177036,-122.147012,NaN
23752,945696,9847407101,4/15/20,1013,PPP,BROWNING AND BROWNING INC,2145 hwy 101,PORT ANGELES,WA,98363.0,...,999 Other,4842 Specialized Freight Trucking,4842.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2145 hwy 101, PORT ANGELES, WA",48.106684,-123.398364,NaN
23753,945697,9939018302,1/31/21,1013,PPS,CMP CONSTRUCTION,4720 158th Pl SW,Lynnwood,WA,98087.0,...,999 Other,2361 Residential Building Construction,2361.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4720 158th Pl SW, Lynnwood, WA",47.855326,-122.297679,NaN


In [4]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [5]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
4219,5492258309,"4205 9th Ave NW, Seattle, WA",NaN,NaN
15222,8327067707,"4500 15TH AVE S, SEATTLE, WA",NaN,NaN
15784,6060237406,"400 E Pine St Ste 322, SEATTLE, WA",NaN,NaN
21085,9777148302,"1037 NE 65th St # 360, Seattle, WA",NaN,NaN


In [6]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,5791407702,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559
1,1101807307,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801
2,1351807201,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385
3,2545378910,"921 Terry Ave, Seattle, WA",47.608549,-122.326329
4,2664207106,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/WA_fips_scraped.csv")


In [ ]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/WA_fips_scraped.csv") 
fips_check_scraped.head() 

In [ ]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [ ]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.count() 

<h4>FIPS Script

In [7]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [11]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('53')] #WA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


  0%|          | 0/23751 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [12]:
geo_id_list =geo_id 

Length Check

In [13]:
print(len(geo_id_list))

23751


Matching FIPS to DataFrame

In [14]:
#for geocode file
fips_fill = fips_check_notna
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:1210]

In [15]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_11607/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
0,5791407702,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,[530330062002034]
1,1101807307,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,[530610410004001]
2,1351807201,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,[530330323193008]
3,2545378910,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,[530330083001012]
4,2664207106,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,[530330230003002]
...,...,...,...,...,...
23750,9825088608,"6346 Rainier Ave S, Seattle, WA",47.545071,-122.274353,[530330111022002]
23751,9830848507,"6021 199th St NE Ste A, Arlington, WA",48.177036,-122.147012,[530610535091000]
23752,9847407101,"2145 hwy 101, PORT ANGELES, WA",48.106684,-123.398364,[530090012003011]
23753,9939018302,"4720 158th Pl SW, Lynnwood, WA",47.855326,-122.297679,[530610501023001]


Remove brackets

In [16]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_11607/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
0,5791407702,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,530330062002034
1,1101807307,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,530610410004001
2,1351807201,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,530330323193008
3,2545378910,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,530330083001012
4,2664207106,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,530330230003002
...,...,...,...,...,...
23750,9825088608,"6346 Rainier Ave S, Seattle, WA",47.545071,-122.274353,530330111022002
23751,9830848507,"6021 199th St NE Ste A, Arlington, WA",48.177036,-122.147012,530610535091000
23752,9847407101,"2145 hwy 101, PORT ANGELES, WA",48.106684,-123.398364,530090012003011
23753,9939018302,"4720 158th Pl SW, Lynnwood, WA",47.855326,-122.297679,530610501023001


In [17]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
12145,9743267004,"503 S. Hilltop Lane, FAIRFIELD, WA",39.012144,-89.801411,NaN
19482,6790008405,"515 KUWILI ST UNIT E, HONOLULU, WA",21.317822,-157.868017,NaN
20315,2243158309,"8960 STATE ROUTE 2 STE 146, Monroe, WA",45.897426,-86.912018,NaN
20321,5003828303,"4 N Park St, Saint John, WA",43.067929,-89.401266,NaN
22878,9978467707,"1316 J STREET, WASCO, WA",35.587474,-119.331151,NaN


Parse for Merge

In [18]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,5791407702,530330062002034
1,1101807307,530610410004001
2,1351807201,530330323193008
3,2545378910,530330083001012
4,2664207106,530330230003002
...,...,...
23750,9825088608,530330111022002
23751,9830848507,530610535091000
23752,9847407101,530090012003011
23753,9939018302,530610501023001


Read in base data

In [19]:
#if from geocode file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_wa.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

#if from FIPS file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/HI_fips_scraped.csv") 
#fips_check = fips_check.drop([ "Unnamed: 0.1.1.1.1.1.1.1"], axis =1) 
#fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN
1,921945,921945,921945,1101807307,4/28/20,1013,PPP,COMPASS HEALTH,4526 FEDERAL AVE Building 4E,EVERETT,...,999 Other,6214 Outpatient Care Centers,6214.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,NaN
2,921946,921946,921946,1351807201,4/15/20,1013,PPP,NORTHWEST RESTAURANTS INC,18815 139th Ave NE Suite C,WOODINVILLE,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,NaN
3,921947,921947,921947,2545378910,4/27/21,1013,PPP,BLOODWORKS,921 Terry Ave,Seattle,...,999 Other,6219 Other Ambulatory Health Care Services,6219.0,SEATTLE DISTRICT OFFICE,e $5-10 million,250 to 499,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,NaN
4,921948,921948,921948,2664207106,4/11/20,1013,PPP,"MOD SUPER FAST PIZZA, LLC",2035 158th Court NE 0.0,Bellevue,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23750,945694,945694,945694,9825088608,3/26/21,1013,PPS,SOUND OIL CO,6346 Rainier Ave S,Seattle,...,999 Other,4543 Direct Selling Establishments,4543.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"6346 Rainier Ave S, Seattle, WA",47.545071,-122.274353,NaN
23751,945695,945695,945695,9830848507,3/12/21,1013,PPS,AEROTECH COATINGS LLC,6021 199th St NE Ste A,Arlington,...,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6021 199th St NE Ste A, Arlington, WA",48.177036,-122.147012,NaN
23752,945696,945696,945696,9847407101,4/15/20,1013,PPP,BROWNING AND BROWNING INC,2145 hwy 101,PORT ANGELES,...,999 Other,4842 Specialized Freight Trucking,4842.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2145 hwy 101, PORT ANGELES, WA",48.106684,-123.398364,NaN
23753,945697,945697,945697,9939018302,1/31/21,1013,PPS,CMP CONSTRUCTION,4720 158th Pl SW,Lynnwood,...,999 Other,2361 Residential Building Construction,2361.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4720 158th Pl SW, Lynnwood, WA",47.855326,-122.297679,NaN


Merge with MainFrame

In [20]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,...,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN,530330062002034
1,921945,921945,921945,1101807307,4/28/20,1013,PPP,COMPASS HEALTH,4526 FEDERAL AVE Building 4E,EVERETT,...,6214 Outpatient Care Centers,6214.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,NaN,530610410004001
2,921946,921946,921946,1351807201,4/15/20,1013,PPP,NORTHWEST RESTAURANTS INC,18815 139th Ave NE Suite C,WOODINVILLE,...,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,NaN,530330323193008
3,921947,921947,921947,2545378910,4/27/21,1013,PPP,BLOODWORKS,921 Terry Ave,Seattle,...,6219 Other Ambulatory Health Care Services,6219.0,SEATTLE DISTRICT OFFICE,e $5-10 million,250 to 499,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,NaN,530330083001012
4,921948,921948,921948,2664207106,4/11/20,1013,PPP,"MOD SUPER FAST PIZZA, LLC",2035 158th Court NE 0.0,Bellevue,...,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,NaN,530330230003002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23750,945694,945694,945694,9825088608,3/26/21,1013,PPS,SOUND OIL CO,6346 Rainier Ave S,Seattle,...,4543 Direct Selling Establishments,4543.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"6346 Rainier Ave S, Seattle, WA",47.545071,-122.274353,NaN,530330111022002
23751,945695,945695,945695,9830848507,3/12/21,1013,PPS,AEROTECH COATINGS LLC,6021 199th St NE Ste A,Arlington,...,3364 Aerospace Product and Parts Manufacturing,3364.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6021 199th St NE Ste A, Arlington, WA",48.177036,-122.147012,NaN,530610535091000
23752,945696,945696,945696,9847407101,4/15/20,1013,PPP,BROWNING AND BROWNING INC,2145 hwy 101,PORT ANGELES,...,4842 Specialized Freight Trucking,4842.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2145 hwy 101, PORT ANGELES, WA",48.106684,-123.398364,NaN,530090012003011
23753,945697,945697,945697,9939018302,1/31/21,1013,PPS,CMP CONSTRUCTION,4720 158th Pl SW,Lynnwood,...,2361 Residential Building Construction,2361.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4720 158th Pl SW, Lynnwood, WA",47.855326,-122.297679,NaN,530610501023001


<b>Fill FIPS Columns - if FIPS file

In [21]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,...,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN,530330062002034,530330062002034
1,921945,921945,921945,1101807307,4/28/20,1013,PPP,COMPASS HEALTH,4526 FEDERAL AVE Building 4E,EVERETT,...,6214.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,NaN,530610410004001,530610410004001
2,921946,921946,921946,1351807201,4/15/20,1013,PPP,NORTHWEST RESTAURANTS INC,18815 139th Ave NE Suite C,WOODINVILLE,...,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,NaN,530330323193008,530330323193008
3,921947,921947,921947,2545378910,4/27/21,1013,PPP,BLOODWORKS,921 Terry Ave,Seattle,...,6219.0,SEATTLE DISTRICT OFFICE,e $5-10 million,250 to 499,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,NaN,530330083001012,530330083001012
4,921948,921948,921948,2664207106,4/11/20,1013,PPP,"MOD SUPER FAST PIZZA, LLC",2035 158th Court NE 0.0,Bellevue,...,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,NaN,530330230003002,530330230003002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23750,945694,945694,945694,9825088608,3/26/21,1013,PPS,SOUND OIL CO,6346 Rainier Ave S,Seattle,...,4543.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"6346 Rainier Ave S, Seattle, WA",47.545071,-122.274353,NaN,530330111022002,530330111022002
23751,945695,945695,945695,9830848507,3/12/21,1013,PPS,AEROTECH COATINGS LLC,6021 199th St NE Ste A,Arlington,...,3364.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6021 199th St NE Ste A, Arlington, WA",48.177036,-122.147012,NaN,530610535091000,530610535091000
23752,945696,945696,945696,9847407101,4/15/20,1013,PPP,BROWNING AND BROWNING INC,2145 hwy 101,PORT ANGELES,...,4842.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2145 hwy 101, PORT ANGELES, WA",48.106684,-123.398364,NaN,530090012003011,530090012003011
23753,945697,945697,945697,9939018302,1/31/21,1013,PPS,CMP CONSTRUCTION,4720 158th Pl SW,Lynnwood,...,2361.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4720 158th Pl SW, Lynnwood, WA",47.855326,-122.297679,NaN,530610501023001,530610501023001


Drop Extra FIPS Columns

In [22]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,530330062002034
1,921945,921945,921945,1101807307,4/28/20,1013,PPP,COMPASS HEALTH,4526 FEDERAL AVE Building 4E,EVERETT,...,999 Other,6214 Outpatient Care Centers,6214.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,530610410004001
2,921946,921946,921946,1351807201,4/15/20,1013,PPP,NORTHWEST RESTAURANTS INC,18815 139th Ave NE Suite C,WOODINVILLE,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,530330323193008
3,921947,921947,921947,2545378910,4/27/21,1013,PPP,BLOODWORKS,921 Terry Ave,Seattle,...,999 Other,6219 Other Ambulatory Health Care Services,6219.0,SEATTLE DISTRICT OFFICE,e $5-10 million,250 to 499,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,530330083001012
4,921948,921948,921948,2664207106,4/11/20,1013,PPP,"MOD SUPER FAST PIZZA, LLC",2035 158th Court NE 0.0,Bellevue,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,530330230003002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23750,945694,945694,945694,9825088608,3/26/21,1013,PPS,SOUND OIL CO,6346 Rainier Ave S,Seattle,...,999 Other,4543 Direct Selling Establishments,4543.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"6346 Rainier Ave S, Seattle, WA",47.545071,-122.274353,530330111022002
23751,945695,945695,945695,9830848507,3/12/21,1013,PPS,AEROTECH COATINGS LLC,6021 199th St NE Ste A,Arlington,...,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6021 199th St NE Ste A, Arlington, WA",48.177036,-122.147012,530610535091000
23752,945696,945696,945696,9847407101,4/15/20,1013,PPP,BROWNING AND BROWNING INC,2145 hwy 101,PORT ANGELES,...,999 Other,4842 Specialized Freight Trucking,4842.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2145 hwy 101, PORT ANGELES, WA",48.106684,-123.398364,530090012003011
23753,945697,945697,945697,9939018302,1/31/21,1013,PPS,CMP CONSTRUCTION,4720 158th Pl SW,Lynnwood,...,999 Other,2361 Residential Building Construction,2361.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4720 158th Pl SW, Lynnwood, WA",47.855326,-122.297679,530610501023001


In [24]:
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1","Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
4219,926163,926163,926163,5492258309,1/25/21,1013,PPS,THE PRODUCTION NETWORK INC.,4205 9th Ave NW,Seattle,...,999 Other,5619 Other Support Services,5619.0,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"4205 9th Ave NW, Seattle, WA",NaN,NaN,NaN
12145,934089,934089,934089,9743267004,4/9/20,1094,PPP,FAIRFIELD CARE,503 S. Hilltop Lane,FAIRFIELD,...,999 Other,6233 Continuing Care Retirement Communities an...,6233.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",20 to 49,"503 S. Hilltop Lane, FAIRFIELD, WA",39.012144,-89.801411,NaN
15222,937166,937166,937166,8327067707,5/1/20,1013,PPP,MACPHERSON'S FRUIT &AMP; PRODUCE,4500 15TH AVE S,SEATTLE,...,999 Other,4452 Specialty Food Stores,4452.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"4500 15TH AVE S, SEATTLE, WA",NaN,NaN,NaN
15784,937728,937728,937728,6060237406,5/13/20,1013,PPP,GREATER SEATTLE BUSINESS ASSOCIATION,400 E Pine St Ste 322,SEATTLE,...,999 Other,4539 Other Miscellaneous Store Retailers,4539.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"400 E Pine St Ste 322, SEATTLE, WA",NaN,NaN,NaN
19482,941426,941426,941426,6790008405,2/11/21,1013,PPS,TIMBERLAND HAWAII INC,515 KUWILI ST UNIT E,HONOLULU,...,999 Other,4441 Building Material and Supplies Dealers,4441.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"515 KUWILI ST UNIT E, HONOLULU, WA",21.317822,-157.868017,NaN
20315,942259,942259,942259,2243158309,1/20/21,1013,PPS,SECOND THAWT INC.,8960 STATE ROUTE 2 STE 146,Monroe,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"8960 STATE ROUTE 2 STE 146, Monroe, WA",45.897426,-86.912018,NaN
20321,942265,942265,942265,5003828303,1/23/21,1094,PPS,INLAND EMPIRE MILLING CO,4 N Park St,Saint John,...,999 Other,1151 Support Activities for Crop Production,1151.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",10 to 19,"4 N Park St, Saint John, WA",43.067929,-89.401266,NaN
21085,943029,943029,943029,9777148302,1/31/21,1013,PPS,LIFFFT INC.,1037 NE 65th St # 360,Seattle,...,999 Other,5415 Computer Systems Design and Related Services,5415.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1037 NE 65th St # 360, Seattle, WA",NaN,NaN,NaN
22878,944822,944822,944822,9978467707,5/1/20,1094,PPP,"TULE TRASH COMPANY, LLC",1316 J STREET,WASCO,...,999 Other,5621 Waste Collection,5621.0,SPOKANE BRANCH OFFICE,"a $150,000-350,000",20 to 49,"1316 J STREET, WASCO, WA",35.587474,-119.331151,NaN


Convert to File

In [25]:
fips_WA = fips_merge_drop


In [26]:
fips_WA.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/WA_fips_scraped.csv") 

Review & Compare

In [27]:
fips_WA = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/WA_fips_scraped.csv") 
fips_WA

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,5.303301e+14
1,1,921945,921945,921945,1101807307,4/28/20,1013,PPP,COMPASS HEALTH,4526 FEDERAL AVE Building 4E,...,999 Other,6214 Outpatient Care Centers,6214.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,5.306104e+14
2,2,921946,921946,921946,1351807201,4/15/20,1013,PPP,NORTHWEST RESTAURANTS INC,18815 139th Ave NE Suite C,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,5.303303e+14
3,3,921947,921947,921947,2545378910,4/27/21,1013,PPP,BLOODWORKS,921 Terry Ave,...,999 Other,6219 Other Ambulatory Health Care Services,6219.0,SEATTLE DISTRICT OFFICE,e $5-10 million,250 to 499,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,5.303301e+14
4,4,921948,921948,921948,2664207106,4/11/20,1013,PPP,"MOD SUPER FAST PIZZA, LLC",2035 158th Court NE 0.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,5.303302e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23750,23750,945694,945694,945694,9825088608,3/26/21,1013,PPS,SOUND OIL CO,6346 Rainier Ave S,...,999 Other,4543 Direct Selling Establishments,4543.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"6346 Rainier Ave S, Seattle, WA",47.545071,-122.274353,5.303301e+14
23751,23751,945695,945695,945695,9830848507,3/12/21,1013,PPS,AEROTECH COATINGS LLC,6021 199th St NE Ste A,...,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6021 199th St NE Ste A, Arlington, WA",48.177036,-122.147012,5.306105e+14
23752,23752,945696,945696,945696,9847407101,4/15/20,1013,PPP,BROWNING AND BROWNING INC,2145 hwy 101,...,999 Other,4842 Specialized Freight Trucking,4842.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2145 hwy 101, PORT ANGELES, WA",48.106684,-123.398364,5.300900e+14
23753,23753,945697,945697,945697,9939018302,1/31/21,1013,PPS,CMP CONSTRUCTION,4720 158th Pl SW,...,999 Other,2361 Residential Building Construction,2361.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4720 158th Pl SW, Lynnwood, WA",47.855326,-122.297679,5.306105e+14
